In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import numpy as np

In [4]:
from librep.datasets.har.loaders import MotionSense_BalancedView20HZ

2022-11-08 19:08:04.574415: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 19:08:04.574435: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
from librep.transforms.topo_ae import (
    TopologicalDimensionalityReduction,
    CustomTopoDimRedTransform
)
from librep.metrics.dimred_evaluator import DimensionalityReductionQualityReport
from Experiments_topoae_KuHar20Hz_helper import *

In [6]:
# from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
# from librep.metrics.report import ClassificationReport
# from librep.estimators import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier

# from librep.transforms.topo_ae import TopologicalDimensionalityReduction
# from librep.estimators.ae.torch.models.topological_ae.topological_ae import TopologicallyRegularizedAutoencoder


# from librep.transforms import UMAP
# from tabulate import tabulate

In [7]:
# MotionSense Loader
loader = MotionSense_BalancedView20HZ(
    root_dir="../../../data/views/MotionSense/balanced_view_20Hz_with_gravity_9.81_acc_standard", 
    download=False
)

# Print the readme (optional)
loader.print_readme()
# kuhar_data = obtainKuHar20Hz()
# train_HD = kuhar_data['train_HD']
# train_LD = kuhar_data['train_LD']
# train_Y = kuhar_data['train_Y']
# test_HD = kuhar_data['test_HD']
# test_LD = kuhar_data['test_LD']
# test_Y = kuhar_data['test_Y']

# Balanced MotionSense View Resampled to 20Hz with Gravity - Multiplied acc by 9.81m/s²

This is a view from [MotionSense] that was spllited into 3s windows and was resampled to 20Hz using the [FFT method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.resample.html#scipy.signal.resample). 

The data was first splitted in three sets: train, validation and test. Each one with the following proportions:
- Train: 70% of samples
- Validation: 10% of samples
- Test: 20% of samples

After splits, the datasets were balanced in relation to the activity code column, that is, each subset have the same number of activitiy samples.

**NOTE**: Each subset contain samples from distinct users, that is, samples of one user belongs exclusivelly to one of three subsets.

## Activity codes
- 0: downstairs (569 train, 101 validation, 170 test) 
- 1: upstairs (569 train, 101 validation, 170 test) 
- 2: sitting (569 train, 101 validation, 170 test) 
- 3: standing (569 train, 101 validation, 170 test) 
- 4: walking (569 train, 101 validation, 170 test) 
- 5: jogging (569 train, 101 validation, 170 test) 
 

## Standartized activity codes
- 0: sit (569 train, 101 validation, 170 test) 
- 1: stand (569 train, 101 validation, 170 test) 
- 2: walk (569 train, 101 validation, 170 test) 
- 3: stair up (569 train, 101 validation, 170 test) 
- 4: stair down (569 train, 101 validation, 170 test) 
- 5: run (569 train, 101 validation, 170 test) 
      




# Preparing Table

Columns:
* RF (Accuracy, F1)
* SVC (Accuracy, F1)
* KNN (Accuracy, F1)
* Trustworthiness
* Continuity
* Co-k-nearest-neighbor-size

Rows:
* UMAP (2 dim)
* Generic Autoencoders
* Topological autoencoders (L=1)
* Topological autoencoders (L=100)

In [8]:
# Load the dataset
# If concat_train_validation is true, return a tuple (train+validation, test)
train_val, test = loader.load(concat_train_validation=True, label=loader.standard_label)
train_val, test

(PandasMultiModalDataset: samples=4020, features=360, no. window=6, label_columns='standard activity code',
 PandasMultiModalDataset: samples=1020, features=360, no. window=6, label_columns='standard activity code')

In [9]:
train_HD = np.array(train_val[:][0])
train_Y = np.array(train_val[:][1])
test_HD = np.array(test[:][0])
test_Y = np.array(test[:][1])

In [10]:
print(np.min(train_val[:][0]), np.max(train_val[:][0]))
print(np.min(test[:][0]), np.max(test[:][0]))

-49.37072418962073 61.15645404569216
-46.928394719326405 60.69211608524809


# Aplicar FFT

In [11]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(
    transforms=[fft_transform],
    new_window_name_prefix="fft."
)

In [12]:
train_dataset_fft = transformer(train_val)
# validation_dataset_fft = transformer(validation_dataset)
test_dataset_fft = transformer(test)

In [13]:
print(train_dataset_fft.X.shape)
# print(validation_dataset_fft.X.shape)
print(test_dataset_fft.X.shape)

(4020, 180)
(1020, 180)


In [14]:
print(np.max(train_dataset_fft.X))

797.3368276806895


In [15]:
train_dataset_fft.X

array([[117.47446671,   7.69616708,   9.12345317, ...,   1.58565333,
          1.92709353,   2.56103481],
       [ 56.99305528,   7.03810632,  21.13174295, ...,   3.59397955,
          1.24491107,   2.19807336],
       [ 43.67148129,   9.31303897,  48.54955013, ...,   0.20668447,
          2.1782541 ,   1.17453151],
       ...,
       [ 48.49117037,   5.91621941,  13.46303591, ...,   0.95960646,
          3.65355504,   4.60541355],
       [ 88.69039888,  27.8862055 ,  19.11225658, ...,   1.48263297,
          3.9119881 ,   2.55326373],
       [ 55.520648  ,   2.58290685,   8.95693627, ...,   5.4340967 ,
          4.2408613 ,  10.63953696]])

In [16]:
train_HD = train_dataset_fft.X
train_LD = None
train_Y = train_dataset_fft.y
test_HD = test_dataset_fft.X
test_LD = None
test_Y = test_dataset_fft.y

In [17]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

(4020, 180) (4020,) (1020, 180) (1020,)


In [18]:
print(np.min(train_HD), np.max(train_HD))
print(np.min(test_HD), np.max(test_HD))

2.983111398280336e-05 797.3368276806895
1.2681145826193632e-05 787.0749219958633


# Visualization helper

In [19]:
# 0: sit (569 train, 101 validation, 170 test)
# 1: stand (569 train, 101 validation, 170 test)
# 2: walk (569 train, 101 validation, 170 test)
# 3: stair up (569 train, 101 validation, 170 test)
# 4: stair down (569 train, 101 validation, 170 test)
# 5: run (569 train, 101 validation, 170 test)
def visualize(X, Y):
    labels = ['sit', 'stand', 'walk', 'stair up', 'stair down', 'run']
    df = pd.DataFrame(dict(x=X[:,0], y=X[:,1], label=Y))
    groups = df.groupby('label')
    fig, ax = plt.subplots()
    ax.margins(0.05)
    for name, group in groups:
        ax.plot(group.x, group.y, marker='.', linestyle='', ms=8, label=labels[name])
    # ax.legend()
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

# Applying Scaler

MinMaxScaler, MaxAbsScaler and StandardScaler apply the scaling PER FEATURE, which means the distance between points would actually be modified, and so, the ranking as well.
Because of this, a new Scaler is used

In [20]:
# from sklearn import preprocessing

# scaler = preprocessing.MinMaxScaler()
# scaler.fit(train_dataset_fft.X)

In [21]:
# train_HD = scaler.transform(train_dataset_fft.X)
# train_LD = None
# # train_Y = train_dataset_fft.y
# test_HD = scaler.transform(test_dataset_fft.X)
# test_LD = None
# # test_Y = test_dataset_fft.y

In [22]:
# print(np.min(train_HD), np.max(train_HD))
# print(np.min(test_HD), np.max(test_HD))

## Set Reporter

In [23]:
reporter = ClassificationReport(
    use_accuracy=True, 
    use_f1_score=True,
    use_classification_report=False,
    use_confusion_matrix=False,
    plot_confusion_matrix=False
)

# **SECTION:** Exploring Topological AE (lambdas)

In [24]:
lambdas_to_explore = [0, 1, 5, 10, 100, 500]
lambdas_to_explore = [1000]
executions_per_model = 10
executions_per_model = 1

def explore_lambda(train_HD, train_Y, test_HD, test_Y, topoae_lambda, times_to_execute=10):
    result_object = {
        'RF-ACC': [],
        'RF-F1': [],
        'SVC-ACC': [],
        'SVC-F1': [],
        'KNN-ACC': [],
        'KNN-F1': [],
        'Trustworthiness': [],
        'Continuity': [],
        'Co-k-NNs': []   
    }
    for _ in range(times_to_execute):
        model_epc = 600
        topo_reducer = CustomTopoDimRedTransform(
            model_name='DeepAE_custom_dim2',
            model_lambda=topoae_lambda,
            patience=None,
            num_epochs=model_epc,
            from_dim=180,
            to_dim=10
        )
        title_plot = "MotionSense 20Hz\nTopoAE lambda " + str(topoae_lambda)
        topo_reducer.fit(train_HD, train_Y, title_plot=title_plot)
        train_LD = np.reshape(topo_reducer.transform(train_HD), (-1,10))
        test_LD = np.reshape(topo_reducer.transform(test_HD), (-1,10))
        experiments_result = run_experiments(train_HD, train_LD, train_Y, test_HD, test_LD, test_Y)
        metrics_reporter = DimensionalityReductionQualityReport()
        metrics_report = metrics_reporter.evaluate([test_HD, test_LD])
        
        result_object['RF-ACC'].append(experiments_result['RF-ACC'])
        result_object['RF-F1'].append(experiments_result['RF-F1'])
        result_object['SVC-ACC'].append(experiments_result['SVC-ACC'])
        result_object['SVC-F1'].append(experiments_result['SVC-F1'])
        result_object['KNN-ACC'].append(experiments_result['KNN-ACC'])
        result_object['KNN-F1'].append(experiments_result['KNN-F1'])
        
        result_object['Trustworthiness'].append(metrics_report['trustworthiness'])
        result_object['Continuity'].append(metrics_report['continuity'])
        result_object['Co-k-NNs'].append(metrics_report['co k nearest neighbor size'])
    return result_object


In [25]:
plot_object = {
    'RF-ACC': [],
    'RF-F1': [],
    'SVC-ACC': [],
    'SVC-F1': [],
    'KNN-ACC': [],
    'KNN-F1': [],
    'Trustworthiness': [],
    'Continuity': [],
    'Co-k-NNs': []   
}

for lambda_val in lambdas_to_explore:
    lambda_exploration = explore_lambda(
        train_HD, train_Y,
        test_HD, test_Y,
        lambda_val,
        times_to_execute=executions_per_model
    )
    # print(lambda_val, lambda_exploration)
    for obj_property in plot_object:
        plot_object[obj_property].append(lambda_exploration[obj_property])
    
# print(plot_object)

Topologically Regularized DeepAE_custom_dim2
Using python to compute signatures
DeepAE_custom_dim, Input: (1, 180) Inner dim: 10
Epoch:1, P:None, Loss:968.5673, Loss-ae:922.5097, Loss-topo:0.0461
Epoch:2, P:None, Loss:763.1454, Loss-ae:736.6598, Loss-topo:0.0265
Epoch:3, P:None, Loss:948.9246, Loss-ae:945.3133, Loss-topo:0.0036
Epoch:4, P:None, Loss:760.7613, Loss-ae:735.3443, Loss-topo:0.0254
Epoch:5, P:None, Loss:742.8690, Loss-ae:729.1104, Loss-topo:0.0138
Epoch:6, P:None, Loss:732.9295, Loss-ae:723.8460, Loss-topo:0.0091
Epoch:7, P:None, Loss:753.2179, Loss-ae:747.5376, Loss-topo:0.0057
Epoch:8, P:None, Loss:775.3448, Loss-ae:773.2660, Loss-topo:0.0021
Epoch:9, P:None, Loss:719.9686, Loss-ae:715.9608, Loss-topo:0.0040
Epoch:10, P:None, Loss:704.7707, Loss-ae:699.2100, Loss-topo:0.0056
Epoch:11, P:None, Loss:608.1164, Loss-ae:599.6828, Loss-topo:0.0084
Epoch:12, P:None, Loss:465.5195, Loss-ae:457.4541, Loss-topo:0.0081
Epoch:13, P:None, Loss:421.9740, Loss-ae:417.3572, Loss-topo:0.0

KeyboardInterrupt: 

In [ ]:
print(plot_object)